In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

### Leitura dos dados

Nesse lab iremos trabalhar em um dataset que nos trás *ratings* referentes a notas atribuídas por investidores de bitcoins a outros investidores.  Para ilustrar o _rating_ iremos utilizar um grafo representado através de uma matriz de adjacência onde os vértices são os investidores e uma arestas entre representa que os vértice *origem* da aresta atribuiu uma nota ao vértice *destino*, contudo, consideraremos apenas as avaliações com nota maior ou igual a 8.

Antes de começarmos temos que lidar com uma pequena limitação, nos dados originais **cada investidor possui um número único que o identifica e este pode chegar a um valor superior a 6500**, quando **no entanto existem apenas um pouco mais de 5800 investidores nos dados**. Quando limitamos os dados às notas maiores que **8.0** sobram apenas **915** desses investidores .

Dito isto é fácil notar que não é apropriado utilizar o id do investidor que vem no dataset para indexar nossa matriz, para isto criamos algumas funções e variáveis auxiliares para ajudar na indexação.


In [2]:
class PageRank():
    def __init__(self):
        self.N = 915
        self.data_frame = pd.read_csv('dados/soc-sign-bitcoinotc.csv')
        self.data_frame.columns = ['origem', 'destino', 'peso', 'tempo']
        self.indice_original = {}
        self.indice_matriz = {}
        self.numero_de_links = {}
        self.idx = 0
        self.matriz_transferencia = np.zeros((self.N, self.N))
        self.google_matriz = None


    def get_indice_origem_destino(self, origem, destino):
            if origem not in self.indice_matriz:
                self.indice_matriz[origem] = self.idx
                self.indice_original[self.idx] = origem
                origem = self.idx
                self.idx += 1
            else:
                origem = self.indice_matriz[origem]

            if destino not in self.indice_matriz:
                self.indice_matriz[destino] = self.idx
                self.indice_original[self.idx] = destino
                destino = self.idx
                self.idx += 1
            else:
                destino = self.indice_matriz[destino]
            return origem, destino

### Calculando a matriz de adjacência (matriz de transferência)

Agora que já temos nossas funções auxiliares para indexação podemos popular nossa matriz de transferência, para isso iremos definir:

**Importância de um nó (k)**: Quantidade de arestas (com peso diferente de zero) que saem do nó.

**Transferência de importância**: Cada nó origem qualquer *X* transfere **1/k** de importância para cada um de seus nós destino. Em termos estatísticos, existe uma probabilidade **1/k** de que um usuário navegando na página *X* visite qualquer uma das páginas que originam em *X*. 

Tendo em mente as definições acima seguimos a seguinte lógica para criar a matriz de transferência :

Se existe um vértice **i** que aponta para um vértice **J** dizemos que 

1. *M[ J ][ i ] = 1 / d [ i ]* 

onde M é a nossa matriz de transferência e *d [ i ]* é a quantidade vértices para qual o vértice *i* aponta.

Porém, como primeiro passo iremos apenas identificar as arestas do grafo e adicionar a nossa matriz com valor igual a **1**.

In [3]:
class PageRank(PageRank):
    def calcula_matriz_transferencia(self):
        for index, row in self.data_frame.iterrows():
            peso = row['peso']
            if peso >= 8.0:
                origem = int(row['origem'])
                destino = int(row['destino'])

                origem, destino = self.get_indice_origem_destino(origem, destino)
                
                if origem not in self.numero_de_links:
                    self.numero_de_links[origem] = 0
                self.numero_de_links[origem] += 1

                self.matriz_transferencia[destino][origem] = 1.0

        self.matriz_transferencia = self.matriz_transferencia
        print("Matriz de transferência")
        print(self.matriz_transferencia)    
        print()

### Normalizando a matriz e eliminando vértices 'sem saída'

Agora que já temos nossa matriz cheia de 1s vamos normaliza-la aplicando a fórmula que foi mencionada acima. Existirá contudo um outro problema, observaremos que existirão colunas na nossa matriz cuja soma será **0**. Isso indica um 'beco sem saída', um vértice que não aponta para nenhum outro.

De um ponto de vista estatístico: Anteriormente consideramos que a probabilidade de que um usuário em um nó qualquer visite uma página 'vizinha' é **1/k**. Porém quando um nó não possui arestas saindo dele, qual é essa probabilidade? O que fazer?

Iremos substituir todos os elementos dessas colunas por **1 / N**, onde N é quantidade de nós do grafo, para indicar que agora existe uma probabilidade igual de a partir desse nó chegar a qualquer outro.

In [4]:
class PageRank(PageRank):
    def normaliza_matriz_transferencia(self):
        cols_to_fix = []

        for i in range(len(self.matriz_transferencia)):
            soma_coluna = 0
            for j in range(len(self.matriz_transferencia[i])):
                soma_coluna += self.matriz_transferencia[j][i]
                if self.matriz_transferencia[j][i] == 1:
                    self.matriz_transferencia[j][i] = 1.0/self.numero_de_links[i] ## Normaliza 
            
             ## Temos um beco sem saida, devemos eliminar
            if soma_coluna == 0:
                cols_to_fix.append(i)

         ## Eliminando os becos sem saída
        for i in cols_to_fix:
            for j in range(len(self.matriz_transferencia)):
                self.matriz_transferencia[j][i] = float(1) / float(self.N)
        print("Matriz de transferência normalizada e sem 'becos sem saída'")
        print(self.matriz_transferencia)
        print()
        self.matriz_transferencia = np.matrix(self.matriz_transferencia)

### Calculando a matriz de PageRank (Google Matrix)

Antes de rodar o algoritmo PageRank em si existem mais um problema que precisa ser tratado:

**Componentes desconexos**: Podem haver componentes desconexos em nosso grafo. Isso faria com que certas partes do grafo ficassem inacessíveis e afetaria também o resultado do algoritmo que acabaria por nos dar rankings imprecisos.

Como resolver? Teleportar! Podemos adicionar ao nosso algoritmo um fator constante **p = 0.15** e dizemos que esta é a probabilidade de teleportar para uma página aleatória do nosso grafo quando estivermos em qualquer nó, mesmo que este não seja um nó solto ou esteja em um componente desconexo. Como as probabilidades devem somar **1** vamos dizer que **0.85** é a chance de de seguir uma aresta qualquer que origine do nó atual.

É válido notar que essa abordagem **converge** ou seja, todos os vértices são visitados em algum momento.

Implementando essa idéia iremos obter uma Google Matrix *M* da forma:

In [5]:
Image(url= "http://pi.math.cornell.edu/~mec/Winter2009/RalucaRemus/Lecture3/Images/M.GIF")

Onde *A* é a nossa matriz de transferência, *p* é a probabilidade de teleporte e *B* é uma matriz de 1s multiplicada por 1/total de nós.

In [6]:
class PageRank(PageRank):

    def get_google_matrix(self):
        prob_teleport = 0.15

        aux_matriz = (float(1) / float(self.N)) * np.ones((self.N, self.N))

        self.google_matriz = (1 - prob_teleport) * self.matriz_transferencia + prob_teleport * aux_matriz
        print('Google Matrix')
        print(self.google_matriz)
        print()

### Rodando o algoritmo

Iniciamos a variável *rank_vector* que irá guardar nossa resposta final com tamanho N (onde N é o número de nós) e cada nó inicia com valor igual a **1/N**. Ao decorrer das iterações do algoritmo esse vetor será multiplicado múltiplas vezes pela nossa *Google Matrix* e os valores nele presentes irão gradualmente mudando, até que a diferença nesses valores sejam tão pequenas que não valha mais a pena continuar iterando. O erro máximo que vamos aceitar é de **0.001**.

Vale salientar ainda que o *rank_vector* é uma distribuição de probabilidades e a soma de seus valores deve ser igual a **1**, para verificar isto faremos algumas verificações mais abaixo.

In [7]:
class PageRank(PageRank):
    def get_page_rank(self, rank_vector = None, n_iter=0, maxerro = 0.001, pages_to_return=5):

        if(rank_vector is None):
            print('Iniciando Algoritmo PageRank...')
            rank_vector = (float(1) / float(self.N)) * np.ones((self.N, 1))

        n_iter += 1

        if sum(abs(self.google_matriz * rank_vector - rank_vector)) > maxerro:
            return self.get_page_rank(self.google_matriz * rank_vector, n_iter)
        else:
            print("Finish! Algoritmo convergiu com {0} iterações para um erro máximo de {1}.".format(n_iter, maxerro))
            print()

            result = self.google_matriz * rank_vector
            result_array = np.array([element[0] for element in np.asarray(result)])
            posicao = 1

            print("Sanity check. A soma das probabilidades deve ser 1, obtivemos: {0}".format(sum([element[0] for element in np.asarray(result)])))
            print()

            for indice in result_array.argsort()[-pages_to_return:][::-1]:
                print("{0}. Vértice id {1} com PageRank {2}".format(posicao, self.indice_original[indice], result_array[indice]))
                posicao += 1

### Resultados

Por fim podemos testar o algoritmo e verificar quais vértices possuem o maior PageRank.

In [8]:
## Testando o algorimo PageRank
x = PageRank()
x.calcula_matriz_transferencia()
x.normaliza_matriz_transferencia()
x.get_google_matrix()
x.get_page_rank()

Matriz de transferência
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Matriz de transferência normalizada e sem 'becos sem saída'
[[0.        0.0010929 0.        ... 0.0010929 0.        0.       ]
 [0.2       0.0010929 0.        ... 0.0010929 0.        0.       ]
 [0.2       0.0010929 0.        ... 0.0010929 0.        0.       ]
 ...
 [0.        0.0010929 0.        ... 0.0010929 0.        0.       ]
 [0.        0.0010929 0.        ... 0.0010929 0.        0.       ]
 [0.        0.0010929 0.        ... 0.0010929 0.        0.       ]]

Google Matrix
[[1.63934426e-04 1.09289617e-03 1.63934426e-04 ... 1.09289617e-03
  1.63934426e-04 1.63934426e-04]
 [1.70163934e-01 1.09289617e-03 1.63934426e-04 ... 1.09289617e-03
  1.63934426e-04 1.63934426e-04]
 [1.70163934e-01 1.09289617e-03 1.63934426e-04 ... 1.09289617e-03
  1.63934426e-04 1.63934426e-04]
 ...
 [1.63934426e-04 1.09289617e-03 1.

### Visualizando os resultados com Gephi

Visualizando o resultado com o Gephi fica evidente que os nós com maior PageRank parecem estar no 'meio'do grafo e tem bastante conexões de entrada e saída com vários outros nós, vejamos:

![title](visu_ri_final.png)


### Conclusões

Caso eu fosse um investidor em bitcoins eu poderia usar essas informações para determinar quem seriam as pessoas ou organizações mais confiáveis e realizar transações com um nível maior de certeza de que não seria vítima de uma fraude/golpe.